In [1]:
import os
import sys
import time
from sentence_transformers import models, losses
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-test", logfilename="../../log/s-bert-test")
device = GPU_info()
seed_everything(111)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert-test_2022-10-21.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
## 평가 설정 parameter 들 ########################################

# 평가할 bert 모델 혹은 sbert모델 경로
model_path = "../../data11/model/sbert/mdistilbertV2.1-sts"
#model_path = "monologg/kobert"


# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = 'eval'
os.makedirs(output_path, exist_ok=True)
csvfilename = 'V3.1.1'   # 유사도 저장할 파일명 (eval/similarity_evaluation_{csffilename}.csv 형식으로 저장)

# 평가 sts 형태의 말뭉치 파일들
use_korsts = 0     #  korsts 파일
use_kluests = 0    # kluests_v1.1 파일
use_glue_sts = 1   # true이면 영문 glue_sts 데이터셋 추가하여 평기 시킴
use_en_sts = 0     # true이면 영문 stsb_multi_mt데이터셋 추가하여 평가시킴.

# * 모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
if use_glue_sts == 1 or use_en_sts == 1:
    do_lower_case = True 
else:
    do_lower_case = False
    
# korsts tsv 파일 경로
korsts_file = '../../data11/korpora/korsts/tune_test.tsv'

# kluests json 파일 경로
kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

eval_batch_size = 64 # 배치파일사이즈 = 크면 eval 시간이 단축됨.단 gpu 메모리 올라감.
max_seq_length=74

In [3]:
from datasets import load_dataset
test_samples = []


# /korsts/tune_test.tsv 파일을 불러옴
if use_korsts == True:
    with open(korsts_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for line in lines:
            s1, s2, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts=[s1,s2], label=score))


# /klue-sts/klue-sts-v1.1_dev.json 파일을 불러옴
if use_kluests == True:
    import json

    with open(kluests_file, "r", encoding='utf-8') as fIn2:
        data = json.load(fIn2)
        for el in data:
            s1 = el["sentence1"]
            s2 = el["sentence2"]
            score = el["labels"]['label']
            test_samples.append(InputExample(texts=[s1,s2], label=score))
        
# 영문 stsb_multi_mt test 버전 파일 로딩함
if use_en_sts == True:
    en_sts_dataset = load_dataset("stsb_multi_mt", name="en", split="test")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
        
if use_glue_sts == True:   
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    
print(test_samples[0:3])
print(f'*test_samples_len: {len(test_samples)}')

[<sentence_transformers.readers.InputExample.InputExample object at 0x7f2f6434a070>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f2e113c2dc0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f2e113c2d60>]
*test_samples_len: 519


In [4]:


# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

logger.info("\n")
logger.info("======================TEST===================")

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)

#model = SentenceTransformer(smodel_path, device='cpu')
#model = SentenceTransformer(smodel_path, cache_folder=output_path)
#model.to(device)
logger.info(f'{model}')

2022-10-21 16:28:12,628 - s-bert-test - INFO - 

2022-10-21 16:28:12,629 - s-bert-test - INFO - ======================TEST===================
2022-10-21 16:28:12,673 - s-bert-test - INFO - SentenceTransformer(
  (0): Transformer({'max_seq_length': 74, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 74, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [5]:
logger.info(f"*main_similarity: {main_similarity}")
logger.info(f"*model path: {model_path}")

if use_korsts == True:
    logger.info(f"*korsts file({korsts_file})")

if use_kluests == True:
    logger.info(f"*klue file({kluests_file})")
    
if use_en_sts == True:
    logger.info(f"*stsb_multi_mt")
  
if use_glue_sts == True:
    logger.info(f"*glue(stsb)")
    
start = time.time()

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, main_similarity=main_similarity, batch_size=eval_batch_size, name=csvfilename, show_progress_bar=True)
result = test_evaluator(model, output_path=output_path)
logger.info(f"\n")
        
logger.info(f'=== result: {result} ===')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info("=====================================================")
logger.info("\n")


2022-10-21 16:28:12,681 - s-bert-test - INFO - *main_similarity: SimilarityFunction.COSINE
2022-10-21 16:28:12,683 - s-bert-test - INFO - *model path: ../../data11/model/sbert/mdistilbertV2.1-sts
2022-10-21 16:28:12,684 - s-bert-test - INFO - *klue file(../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json)


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

2022-10-21 16:28:18,490 - s-bert-test - INFO - 

2022-10-21 16:28:18,491 - s-bert-test - INFO - === result: 0.8275025484180157 ===
2022-10-21 16:28:18,492 - s-bert-test - INFO - === 처리시간: 5.806 초 ===
2022-10-21 16:28:18,492 - s-bert-test - INFO - =====================================================
2022-10-21 16:28:18,493 - s-bert-test - INFO - 

